<a href="https://colab.research.google.com/github/dwihdyn/algo-trading/blob/master/sector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***US STOCKS PERFORMANCE BY SECTOR***

In [0]:
# packages
import pandas as pd
import pandas_datareader as pdr
from datetime import datetime
import matplotlib.pyplot as plt
plt.style.use('seaborn')
import requests
import json

In [0]:
# pull data | RKKWDNO3XD91IUBZ -> EXXJQAB3KQ91VHOM (http://decode.org/)
response = requests.get("https://www.alphavantage.co/query?function=SECTOR&apikey=RKKWDNO3XD91IUBZ")

alphadict = json.loads(response.text)
alphadict.keys() # get keys that need to be typed inside pd.DataFrame

realTime = pd.DataFrame(alphadict['Rank A: Real-Time Performance'], index=['realTime']).T
oneDay = pd.DataFrame(alphadict['Rank B: 1 Day Performance'], index=['oneDay']).T
fiveDay = pd.DataFrame(alphadict['Rank C: 5 Day Performance'], index=['fiveDay']).T
oneMonth = pd.DataFrame(alphadict['Rank D: 1 Month Performance'], index=['oneMonth']).T
threeMonth = pd.DataFrame(alphadict['Rank E: 3 Month Performance'], index=['threeMonth']).T
yearToDate = pd.DataFrame(alphadict['Rank F: Year-to-Date (YTD) Performance'], index=['yearToDate']).T
oneYear = pd.DataFrame(alphadict['Rank G: 1 Year Performance'], index=['oneYear']).T
threeYear = pd.DataFrame(alphadict['Rank H: 3 Year Performance'],index=['threeYear']).T
fiveYear = pd.DataFrame(alphadict['Rank I: 5 Year Performance'], index=['fiveYear']).T
tenYear = pd.DataFrame(alphadict['Rank J: 10 Year Performance'], index=['tenYear']).T

# convert all data from string to float (idea : https://stackoverflow.com/questions/12432663/what-is-a-clean-way-to-convert-a-string-percent-to-a-float)



In [0]:
allData = [realTime, oneDay, fiveDay, oneMonth, threeMonth, yearToDate, oneYear, threeYear, fiveYear, tenYear]


sector = pd.merge(realTime, oneDay, left_index=True, right_index=True)

i = 1
while i < 10:
  sector = pd.merge(sector, allData[i], left_index=True, right_index=True)
  i = i + 1

sector

,realTime,oneDay_x,oneDay_y,fiveDay,oneMonth,threeMonth,yearToDate,oneYear,threeYear,fiveYear,tenYear
Utilities,0.11%,0.11%,0.11%,-0.55%,2.62%,0.29%,-1.27%,22.86%,31.34%,34.23%,105.19%
Industrials,-0.12%,-0.12%,-0.12%,1.10%,3.19%,11.59%,1.69%,28.39%,29.95%,43.95%,187.76%
Consumer Staples,-0.17%,-0.17%,-0.17%,-1.06%,0.76%,4.01%,-0.99%,23.26%,20.45%,28.67%,133.52%
Energy,-0.34%,-0.34%,-0.34%,0.36%,6.32%,10.30%,0.51%,5.99%,-17.26%,-22.13%,6.70%
Communication Services,-0.55%,-0.55%,-0.55%,-0.22%,3.58%,12.06%,0.75%,30.24%,3.62%,19.34%,59.64%
Health Care,-0.85%,-0.85%,-0.85%,-0.96%,3.21%,15.99%,-0.66%,19.51%,48.11%,48.46%,225.86%
Consumer Discretionary,-0.85%,-0.85%,-0.85%,-0.09%,3.88%,7.28%,0.43%,25.58%,52.90%,74.20%,321.36%
Information Technology,-1.06%,-1.06%,-1.06%,0.45%,6.59%,18.06%,0.65%,48.91%,100.72%,134.71%,337.45%
Financials,-1.10%,-1.10%,-1.10%,-0.40%,2.98%,14.39%,-0.17%,28.01%,32.08%,53.12%,163.45%
Materials,-1.62%,-1.62%,-1.62%,-2.75%,0.86%,7.23%,-2.84%,17.87%,20.09%,22.58%,87.62%


***FOR FUN : CHECKING 
MANUAL SMA10 VS SMA10 FROM ALPHAVANTAGE*** 

In [0]:
from IPython.display import display # display mulltiple things in one block

# pull SMA data
response = requests.get("https://www.alphavantage.co/query?function=SMA&symbol=MSFT&interval=daily&time_period=10&series_type=close&apikey=RKKWDNO3XD91IUBZ")
alphadict = json.loads(response.text)
alphadict.keys() # get keys that need to be typed inside pd.DataFrame

# data cleansing
sma10 = pd.DataFrame(alphadict['Technical Analysis: SMA']).T
sma10.index = pd.to_datetime(sma10.index)
sma10 = sma10.sort_index(ascending=True)
sma10.columns = ['SMA10']
sma10 = sma10.astype(float)

# =======================================================================

# pull stock data
response = requests.get("https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=MSFT&apikey=RKKWDNO3XD91IUBZ")
alphadict = json.loads(response.text)
alphadict.keys()

# data cleansing
stock = pd.DataFrame(alphadict['Time Series (Daily)']).T
stock.index = pd.to_datetime(stock.index)
stock = stock.sort_index(ascending=True)
stock.columns = ['open', 'high', 'low', 'close', 'volume']
stock = stock.astype(float)

# calculate SMA from stock data
stock['SMA10'] = stock['close'].rolling(10).mean()


# show both tables
display(stock['SMA10'].tail()) # manual calculation
display(sma10['SMA10'].tail()) # alphavantage

# calculate difference. expected 0
diff = display(round(sma10['SMA10'].tail() - stock['SMA10'].tail()))




2019-12-30    156.772
2019-12-31    156.989
2020-01-02    157.582
2020-01-03    158.007
2020-01-06    158.339
Name: SMA10, dtype: float64

2019-12-30    156.772
2019-12-31    156.989
2020-01-02    157.582
2020-01-03    158.007
2020-01-06    158.339
Name: SMA10, dtype: float64

2019-12-30    0.0
2019-12-31    0.0
2020-01-02    0.0
2020-01-03    0.0
2020-01-06    0.0
Name: SMA10, dtype: float64